In [ ]:
using DrWatson 
@quickactivate "diff_gleam"

Installing ModelingToolkit (was only needed once)

In [ ]:
# using Pkg
# Pkg.add("ModelingToolkit")
# Pkg.add("DifferentialEquations")
# Pkg.add("Plots")

## First simple ODE
$$
\frac{dx}{dt} = \dot x  = \frac{f(t) - x(t)}{\tau}
$$
Start with $f(t) = 1$

Introduction from https://docs.sciml.ai/ModelingToolkit/stable/tutorials/ode_modeling/ 

In [ ]:
using ModelingToolkit

@variables t
D = Differential(t) #differential operator so ../dt

@mtkmodel FOL begin
    @parameters begin
        τ #parameters
    end
    @variables begin
        x(t) #state variables
    end
    @equations begin
        D(x) ~ (1 - x) /τ #not x(t)
        #tilde anlog to = here
    end
end

@mtkbuild fol = FOL()

In [ ]:
typeof(fol)

In [ ]:
a = 1
typeof(a => 3) #way to define a pair!

In [ ]:
using DifferentialEquations
using Plots 

prob = ODEProblem(
    fol, #f = ODE function
    [fol.x => 0.0], #the initial conditions
    (0,10), #tspan
    [fol.τ => 3.0], #parameters
)
plot(solve(prob))

## Algebraic relations and structural simplification

In [ ]:
@mtkmodel FOL begin
    @parameters begin
        τ
    end
    @variables begin
        x(t)
        RHS(t)
    end
    begin
        D = Differential(t)
    end
    @equations begin
        RHS ~ (1 - x)/τ
        D(x) ~ RHS
    end
end

@mtkbuild fol = FOL()

In [ ]:
equations(fol)

In [ ]:
fol.RHS

more info on observed see: https://docs.sciml.ai/ModelingToolkit/stable/internals/#Observables-and-Variable-Elimination

In [ ]:
observed(fol)

You can plot this righthandside! HANDY (think of e.g. plotting Evaporation)

In [ ]:
prob = ODEProblem(fol,
    [fol.x => 0.0],
    (0.0,10),
    [fol.τ => 3.0])
sol = solve(prob)

In [ ]:
sol.t

In [ ]:
plot(sol, idxs = [fol.x, fol.RHS])

In [ ]:
sol[fol.x]

In [ ]:
sol[fol.RHS] #so you can extract this RHS!!

## Time-variable forcing function

In [ ]:
@mtkmodel FOL begin
    @parameters begin
        τ
    end
    @variables begin
        x(t)
        f(t)
        RHS(t)
    end
    begin
        D = Differential(t)
    end
    @equations begin
        f ~ sin(t)
        RHS ~ (f - x)/τ
        D(x) ~ RHS
    end
end

@macroexpand @named fol_varialbe_f = FOL()

In [ ]:
FOL(name = :example_symbol) #equivalent code!

OFTEN occuring scenario: time-series data as forcings!

In [ ]:
value_vector = randn(10)

`t > 10 ? value_vector[end] : value_vector[Int(floor(t))+1*]`
is equivalent to
```
if t > 10
    value_vector[end]
elseif t < 10
    value_vector[Int(floor(t)+1)]
end

```

In [ ]:
# @macroexpand @register_symbolic f_fun(t)

In [ ]:
#the following is an abbreviation of if ... 
f_fun(t) = t >= 10 ? value_vector[end] : value_vector[Int(floor(t))+1]
f_fun.(0:0.5:10)
@register_symbolic f_fun(t)

@mtkmodel FOLex begin
    @parameters begin
        τ
    end
    @variables begin
        x(t)
        f(t)
        RHS(t)
    end
    # @structural_parameters begin
    #     h = 1
    # end
    begin
        D = Differential(t)
    end
    @equations begin
        f ~ f_fun(t)
        RHS ~ (f - x)/τ
        D(x) ~ RHS
    end
end

@mtkbuild fol_external = FOLex()

prob = ODEProblem(
    fol_external,
    [fol_external.x => 0.0],
    (0.0, 10.0),
    [fol_external.τ => 0.75]
)
sol = solve(prob)
plot(sol, idxs = [fol_external.x, fol_external.RHS, fol_external.f])

## Own experiment: other way of implementing

Another way of making ODEsystems! https://docs.sciml.ai/ModelingToolkit/stable/tutorials/programmatically_generating/#programmatically  I think I prefer it this way

In [ ]:
# @variables t
# D = Differential(t)

function test_model(;name) #so name is a keyword argument, you have to provide the key!
    @parameters τ
    @variables x(t) f(t) RHS(t)
    D = Differential(t)
    eqs = [
        f ~ sin(t),
        RHS ~ (f - x)/τ,
        D(x) ~ RHS
    ]
    return ODESystem(eqs; name, defaults = Dict(τ => 0.75))
end
#test_model(name = :test_model)
@named test = test_model()
test = complete(structural_simplify(test))

prob = ODEProblem(test, [test.x => 0.0], (0.0, 10.0))
solution = solve(prob)
plot(solution, idxs = [test.RHS, test.x, test.τ])

## Component based models

So combining multiple models!

FACTORY -> from here MULTIPLE initialisations! 

In [ ]:
function fol_factory(seperate = false; name)  #so name is a non-necessary keyword argument
    @parameters τ
    @variables t x(t) f(t) RHS(t)

    eqs = seperate ? [
        RHS ~ (f - x)/τ,D(x) ~ RHS] : #if seperate is true, then RHS seperate from the ODE
        D(x) ~ (f - x)/τ
    ODESystem(eqs; name)
end 

In [ ]:
fol_1 = fol_factory(name = :fol_1) #symbol as name!

In [ ]:
fol_2 = fol_factory(true, name = :fol_2) #symbol as name!

You can now CONNECT these systems!

In [ ]:
connections = [fol_1.f ~ 1.5, fol_2.f ~ fol_1.x]

In [ ]:
ODESystem(connections, name = :connected) #you must provide a name!

In [ ]:
connected = compose(ODESystem(connections, name = :connected), fol_1, fol_2)

In [ ]:
connected.fol_1

In [ ]:
connected.fol_2

In [ ]:
connected.fol_2.x

The system can be simplified!

In [ ]:
connected_simp = structural_simplify(connected)

https://arxiv.org/ftp/arxiv/papers/2103/2103.05244.pdf

In [ ]:
full_equations(connected_simp) #combines all the info we gave!

In [ ]:
u0 =  [fol_1.x => -0.5,
    fol_2.x => 1.0]
p = [fol_1.τ => 2.0,
    fol_2.τ => 4.0]

prob = ODEProblem(connected_simp, u0, (0.0, 10.0), p)
sol = solve(prob)
plot(solve(prob))

In [ ]:
sol[fol_2.RHS]